In [107]:
# 1. Загрузка библиотек

In [9]:
import requests as rq
from bs4 import BeautifulSoup
import re
import pandas as pd
import time
from random import randint
from tqdm import tqdm

In [10]:
#2. Определение функций сбора, фильтрации и форматирования

In [11]:
news_links = []

def get_news_links(start_page, end_page):

    re_links=[]
    for it in tqdm(range(start_page,end_page)):
        temp_url = "https://www.securitylab.ru/news/page1_" + str(it) + ".php"
        temp_page = rq.get(temp_url)
        temp_page.encoding = "UTF-8"
        soup=BeautifulSoup(temp_page.text, features = "html.parser")
        for link in soup.find_all("a"):
            for i in re.findall(r"news/\d+", str(link.get("href"))):
                if len(str(i)) > 1:
                    re_links.append(str(i))
                else:
                    pass
        time.sleep(randint(2,4))    
    for i in re_links:
        news_links.append("https://www.securitylab.ru/"+i+".php")

In [12]:
texts = []
titles = []
subtitles = []
tags = []
dates = []

def get_all(news_links):
    for i in tqdm(range(len(news_links))):
        temp_tags = []
        temp_url = news_links[i]
        temp_page = rq.get(temp_url)
        temp_soup = BeautifulSoup(temp_page.text, "html.parser")
        titles.append(temp_soup.title.text.strip())
        subtitles.append(temp_soup.p.text.strip())
        for i in temp_soup.find_all("a", class_ = "tag tag-outline-primary"):
            temp_tags.append(i.text.strip())
        temp_tags=list(set(temp_tags))
        tags.append(tuple(temp_tags))
        temp_string=""
        temp_string += temp_soup.find("div", itemprop = "description").text.strip()
        texts.append(temp_string)
        dates.append(temp_soup.time["datetime"].strip())
        time.sleep(randint(2,4))
        
    return texts, titles, subtitles, tags, dates

In [13]:
filtered_texts =[]
filtered_tags = []
filtered_titles = []
filtered_subtitles = []
filtered_news_links = []

def filter(texts, titles, subtitles, tags, news_links):
    for i in texts:
        if type(i) == str:
            filtered_texts.append(texts[texts.index(i)])
            filtered_titles.append(titles[texts.index(i)])
            filtered_subtitles.append(subtitles[texts.index(i)])
            filtered_tags.append(tags[texts.index(i)])
            filtered_news_links.append(news_links[texts.index(i)])
    return filtered_texts, filtered_titles, filtered_subtitles, filtered_tags, filtered_news_links

In [14]:
clear_texts=[]

def clear(filtered_texts):
    for i in filtered_texts:
        t=re.sub(r"(\r\n\s*\t)","",i)
        ct=re.sub(r"\s{2,10}|\r+|\n+|\t+"," ",t)
        clear_texts.append(ct)
    return clear_texts

In [15]:
clear_dates = []

def clear_date(dates):
    for i in dates:
        clear=re.sub(r"T.+","",i)
        clear_dates.append(clear)
    return clear_dates

In [28]:
def save_df(name, clear_dates, filtered_titles, filtered_subtitles, filtered_tags, clear_texts):

    data_list = clear_dates, filtered_titles, filtered_subtitles, filtered_tags, clear_texts

    SecurityLab_df = pd.DataFrame(data_list, columns = filtered_news_links).T
    SecurityLab_df = SecurityLab_df.rename(columns={0:"Date", 1:"Title", 2:"Subtitle", 3:"Tags", 4:"Text"})
    SecurityLab_df.to_csv(name+'.csv', encoding='utf-8')

In [ ]:
#3. Сбор содержимого, фильтрация и сохранение

In [17]:
get_news_links(1,406)

100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:10<00:00,  3.61s/it]


In [18]:
get_all(news_links)

100%|██████████████████████████████████████████████████████████████████████████████████| 45/45 [02:38<00:00,  3.52s/it]


(['Хакерский рынок Genesis Market, на котором продавались украденные аккаунты Netflix, Amazon и других сервисов, как ни в чём не бывало   продолжает работать  , несмотря на заявления полиции о его ликвидации.\r\n  \n\r\n  \t В прошлом месяце ФБР   торжественно объявила  , что инфраструктура киберпреступного рынка Genesis Market был ликвидирована, а сам маркет был удалён из публичного доступа в общедоступном интернете. Однако идентичная версия рынка, размещенная в тёмном сегменте интернета, даркнете, остается активной до сих пор.\r\n  \n\r\n  \t Администраторы площадки только «подлили масла в огонь», когда разместили в шапке сайта информацию о том, что маркет «полностью функционирует», побуждая потенциальных покупателей не бояться приобретать необходимые данные и услуги.\r\n  \n\r\n  \t Genesis Market описывается полицией как «опасный» веб-сайт, специализирующийся на продаже логинов, IP-адресов и cookie-файлов, составляющих «цифровые отпечатки» жертв. На момент закрытия общедоступного в

In [19]:
filter(texts, titles, subtitles, tags, news_links)

(['Хакерский рынок Genesis Market, на котором продавались украденные аккаунты Netflix, Amazon и других сервисов, как ни в чём не бывало   продолжает работать  , несмотря на заявления полиции о его ликвидации.\r\n  \n\r\n  \t В прошлом месяце ФБР   торжественно объявила  , что инфраструктура киберпреступного рынка Genesis Market был ликвидирована, а сам маркет был удалён из публичного доступа в общедоступном интернете. Однако идентичная версия рынка, размещенная в тёмном сегменте интернета, даркнете, остается активной до сих пор.\r\n  \n\r\n  \t Администраторы площадки только «подлили масла в огонь», когда разместили в шапке сайта информацию о том, что маркет «полностью функционирует», побуждая потенциальных покупателей не бояться приобретать необходимые данные и услуги.\r\n  \n\r\n  \t Genesis Market описывается полицией как «опасный» веб-сайт, специализирующийся на продаже логинов, IP-адресов и cookie-файлов, составляющих «цифровые отпечатки» жертв. На момент закрытия общедоступного в

In [20]:
clear(filtered_texts)

['Хакерский рынок Genesis Market, на котором продавались украденные аккаунты Netflix, Amazon и других сервисов, как ни в чём не бывало продолжает работать , несмотря на заявления полиции о его ликвидации. В прошлом месяце ФБР торжественно объявила , что инфраструктура киберпреступного рынка Genesis Market был ликвидирована, а сам маркет был удалён из публичного доступа в общедоступном интернете. Однако идентичная версия рынка, размещенная в тёмном сегменте интернета, даркнете, остается активной до сих пор. Администраторы площадки только «подлили масла в огонь», когда разместили в шапке сайта информацию о том, что маркет «полностью функционирует», побуждая потенциальных покупателей не бояться приобретать необходимые данные и услуги. Genesis Market описывается полицией как «опасный» веб-сайт, специализирующийся на продаже логинов, IP-адресов и cookie-файлов, составляющих «цифровые отпечатки» жертв. На момент закрытия общедоступного веб-сайта Genesis Market там было продано более двух мил

In [21]:
clear_date(dates)

['2023-05-15',
 '2023-05-15',
 '2023-05-15',
 '2023-05-15',
 '2023-05-15',
 '2023-05-15',
 '2023-05-15',
 '2023-05-15',
 '2023-05-15',
 '2023-05-15',
 '2023-05-14',
 '2023-05-14',
 '2023-05-14',
 '2023-05-14',
 '2023-05-14',
 '2023-05-14',
 '2023-05-14',
 '2023-05-13',
 '2023-05-13',
 '2023-05-13',
 '2023-05-12',
 '2023-05-12',
 '2023-05-12',
 '2023-05-12',
 '2023-05-12',
 '2023-05-12',
 '2023-05-12',
 '2023-05-12',
 '2023-05-12',
 '2023-05-12',
 '2023-05-12',
 '2023-05-12',
 '2023-05-12',
 '2023-05-12',
 '2023-05-12',
 '2023-05-12',
 '2023-05-12',
 '2023-05-12',
 '2023-05-12',
 '2023-05-12',
 '2023-05-11',
 '2023-05-11',
 '2023-05-11',
 '2023-05-11',
 '2023-05-11']

In [29]:
save_df("SecurityLab", clear_dates, filtered_titles, filtered_subtitles, filtered_tags, clear_texts)